## import

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
from healpy.sphtfunc import map2alm, alm2cl

%config InlineBackend.figure_format = 'retina'                                    # so you can see plots in HD :) 
#colors = sns.color_palette("colorblind").as_hex()
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

## extract CMB maps and masks

In [2]:
def extract_data(filepath, hdu=1):
    '''
    Extracts input HDU from map Planck FITS file (we only need the first one)
    Document function once fully tested
    '''
    
    hdul = fits.open(filepath)
    hdul.info()                              # prints metadata about FITS file (HDUs)
    print('='*90)
    
    data = hdul[hdu].data                    # hdul is a list of HDU objects
    
    return data

# Define each filepath
map_hm1 = 'data/HFI_SkyMap_143_2048_R3.01_halfmission-1.fits'        # map for half mission 1 143 GHZ
map_hm2 = 'data/HFI_SkyMap_143_2048_R3.01_halfmission-2.fits'        # map for half mission 2 143 GHZ
mask_gp = 'data/HFI_Mask_GalPlane-apo5_2048_R2.00.fits'              # galactic plane mask
mask_ps = 'data/HFI_Mask_PointSrc_2048_R2.00.fits'                   # mask point source

# Extract maps from half missions 1 and 2, and masks for galactic plane and point source
data_map_hm1 = extract_data(map_hm1)
data_map_hm2 = extract_data(map_hm2)
data_mask_gp = extract_data(mask_gp)
data_mask_ps = extract_data(mask_ps)

Filename: data/HFI_SkyMap_143_2048_R3.01_halfmission-1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  FREQ-MAP      1 BinTableHDU     75   50331648R x 10C   [E, E, E, J, E, E, E, E, E, E]   
Filename: data/HFI_SkyMap_143_2048_R3.01_halfmission-2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  FREQ-MAP      1 BinTableHDU     75   50331648R x 10C   [E, E, E, J, E, E, E, E, E, E]   
Filename: data/HFI_Mask_GalPlane-apo5_2048_R2.00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  GAL-MASK      1 BinTableHDU     54   50331648R x 8C   [E, E, E, E, E, E, E, E]   
Filename: data/HFI_Mask_PointSrc_2048_R2.00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SRC-INT       1 BinTableHDU     49   50331648

In [ ]:
def find_spherical_harmonics(map_table, gp_table, ps_table,
                             stoke_param ='I_STOKES', galactic_plane='GAL040', frequency=143):
    '''
    Computes the spherical harmonics from a map,
    masking the stokes parameters from the raw CMB data
    Fully document function once fully tested
    '''
    
    # extract stoke parameter (T or I by default) and mask based on galactic plane and point source
    x = map_table[stoke_param]
    gp_mask = gp_table[galactic_plane]
    ps_mask = ps_table[f'F{frequency}']
    x_masked = x*gp_mask*ps_mask
    
    # compute spherical harmonics
    nside = x_masked.shape[0]
    a_lm =  map2alm(x_masked)
    
    return a_lm

# Call function
a_lm_1 = find_spherical_harmonics(data_map_hm1, data_mask_gp, data_mask_ps)
#a_lm_2 = find_spherical_harmonics(data_map_hm2, data_mask_gp, data_mask_ps)

In [ ]:
print(a_lm_1.shape)

In [ ]:
# plot spherical harmonics as a function of the multipole
l = np.arange(a_lm_1.shape[0])

plt.figure(figsize=(13,9))
plt.plot(l, a_lm_1.real)
plt.show()

In [ ]:
# plot spherical harmonics as a function of the multipole
l = np.arange(a_lm_1.shape[0])

plt.figure(figsize=(13,9))
plt.plot(l, np.abs(a_lm_1.real))
plt.show()

In [ ]:
print(a_lm_1.shape)

In [ ]:
12*2048**2

In [ ]:
(3*2048)*(3*2048+1)/2

In [ ]:
## compute 

In [ ]:
# find beam functions
beam_hm1 = 'data/Bl_T_R3.01_fullsky_100hm1x143hm1.fits'
beam_hm2 = 'data/Bl_T_R3.01_fullsky_100hm1x143hm2.fits'

data_beam_hm1 = extract_data(beam_hm1)
data_beam_hm2 = extract_data(beam_hm2)

In [ ]:
print(data_beam_hm1)

In [ ]:
def find_power_spectrum(alm_1, alm_2, nside=50331648,
                        M_ll, p_l, b_l, f_l=1, n_l=0):
    '''
    Finds the cross-power spectrum (via pseudo cross power spectrum) given the 
    coefficients of the spherical harmonics
    Document function once fully tested
    '''
    # compute pseudo power spectrum
    D_l = alm2cl(alm_1, alm_2)
    
    # now define all the instrument related biases
    p_l = pixwin(nside)
    m_ll = #? 
    b_l = #??